In [1]:
from __future__ import print_function

from functools import partial
from six.moves import xrange

from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import pandas as pd
import os
from datetime import datetime, timedelta

# Data Processing helpers

In [2]:
data_path = 'C:/Users/ehniv/OneDrive/SFU/2021 - 2022 Academics/Spring 2022/MATH 402W - Operations Research Clinic/Datasets/Testing Datasets/Processed'

#processes the time matrix
time_matrix = pd.read_csv(data_path+'/20220304_location_903_morning_time_matrix.csv', index_col = 'Unnamed: 0')
processed_time_matrix = time_matrix.values.tolist()

In [3]:
time_slot_data = pd.read_csv(data_path+'/20220304_location_903_morning_time_slot.csv')

#creates a tuple of time data
processed_time_windows = time_slot_data[["start_time_scaled","end_time_scaled"]].apply(tuple, axis = 1).to_list()

In [4]:
addresses = time_slot_data['dummy_add'].tolist()

In [5]:
output_path = os.path.join(data_path,'903_morning_solution')

# Config

In [6]:
###########################
# Problem Data Definition #
###########################
def create_data_model():
    """Stores the data for the problem"""
    data = {}
    # Locations in block unit
#     _locations = \
#             [(4, 4), # depot
#              (2, 0), (8, 0), # locations to visit
#              (0, 1), (1, 1),
#              (5, 2), (7, 2),
#              (3, 3), (6, 3),
#              (5, 5), (8, 5),
#              (1, 6), (2, 6),
#              (3, 7), (6, 7),
#              (0, 8), (7, 8)]
    # Compute locations in meters using the block dimension defined as follow
    # Manhattan average block: 750ft x 264ft -> 228m x 80m
    # here we use: 114m x 80m city block
    # src: https://nyti.ms/2GDoRIe "NY Times: Know Your distance"
#     data['locations'] = [(l[0] * 114, l[1] * 80) for l in _locations]
    
    #Input Data
    data['raw_time_matrix'] = time_matrix
    data['time_windows'] = processed_time_windows
    data['time_matrix'] = processed_time_matrix
    data['num_locations'] = len(data['time_matrix'])
    
    #Parameters
    demand_per_location = 1
    delivery_start_time = '06:45' #earliest vehicle can leave the depot (6:45 or 14:45)
    data['time_per_demand_unit'] = 12  #Service Time At Location = time_per_demand_unit * demand_at_location
    data['num_vehicles'] = 2
    data['vehicle_capacity'] = 14
    data['break_start_time'] = 255
    data['break_end_time'] = 315
    data['break_duration'] = 30
    load_balacing = False
    load_factor = 0.9
    
    
    data['solver_limit'] = 1000
    
    data['address_lookup'] = {i: addresses[i] for i in range(len(addresses))}
    data['time_slot_lookup'] = {i: processed_time_windows[i] for i in range(len(processed_time_windows))}
    
    #assigns address for each driver (ie. if there is 4 drivers, 4 depot nodes are created)
    for index in range(2* data['num_vehicles']):
        data['address_lookup'][len(addresses) + index] = addresses[0] 
        data['time_slot_lookup'][len(processed_time_windows) + index] = processed_time_windows[0]
    
    
    #Auto Populates
    data['demands'] = [0] + [demand_per_location for i in range(data['num_locations'] - 1)]
    data['depot'] = 0
    data['delivery_start_time'] = datetime.strptime(delivery_start_time, '%H:%M')
    if(load_balacing):
        data['vehicle_capacity'] = int(data['vehicle_capacity'] * load_factor)
    
    
    return data

In [7]:
#######################
# Problem Constraints #
#######################
# def manhattan_distance(position_1, position_2):
#     """Computes the Manhattan distance between two points"""
#     return (
#         abs(position_1[0] - position_2[0]) + abs(position_1[1] - position_2[1]))


# def create_distance_evaluator(data):
#     """Creates callback to return distance between points."""
#     _distances = {}
#     # precompute distance between location to have distance callback in O(1)
#     for from_node in xrange(data['num_locations']):
#         _distances[from_node] = {}
#         for to_node in xrange(data['num_locations']):
#             if from_node == to_node:
#                 _distances[from_node][to_node] = 0
#             else:
#                 _distances[from_node][to_node] = (manhattan_distance(
#                     data['locations'][from_node], data['locations'][to_node]))

#     def distance_evaluator(manager, from_node, to_node):
#         """Returns the manhattan distance between the two nodes"""
#         return _distances[manager.IndexToNode(from_node)][manager.IndexToNode(
#             to_node)]

#     return distance_evaluator


def create_demand_evaluator(data):
    """Creates callback to get demands at each location."""
    _demands = data['demands']

    def demand_evaluator(manager, node):
        """Returns the demand of the current node"""
        return _demands[manager.IndexToNode(node)]

    return demand_evaluator


def add_capacity_constraints(routing, data, demand_evaluator_index):
    """Adds capacity constraint"""
    capacity = 'Capacity'
    routing.AddDimension(
        demand_evaluator_index,
        0,  # null capacity slack
        data['vehicle_capacity'],
        True,  # start cumul to zero
        capacity)


def create_time_evaluator(data):
    """Creates callback to get total times between locations."""

    def service_time(data, node):
        """Gets the service time for the specified location."""
        return data['demands'][node] * data['time_per_demand_unit']

    def travel_time(data, from_node, to_node):
        """Gets the travel times between two locations."""
        if from_node == to_node:
            travel_time = 0
        else:
            travel_time = data['time_matrix'][from_node][to_node]
        return travel_time

    _total_time = {}
    # precompute total time to have time callback in O(1)
    for from_node in xrange(data['num_locations']):
        _total_time[from_node] = {}
        for to_node in xrange(data['num_locations']):
            if from_node == to_node:
                _total_time[from_node][to_node] = 0
            else:
                _total_time[from_node][to_node] = int(
                    service_time(data, from_node) + travel_time(
                        data, from_node, to_node))

    def time_evaluator(manager, from_node, to_node):
        """Returns the total time between the two nodes"""
        return _total_time[manager.IndexToNode(from_node)][manager.IndexToNode(
            to_node)]

    return time_evaluator


def add_time_window_constraints(routing, manager, data, time_evaluator_index):
    """Add Global Span constraint"""
    time = 'Time'
    routing.AddDimension(
        time_evaluator_index,
        999999,  # allow waiting time
        999999,  # maximum time per vehicle
        False,  # don't force start cumul to zero since we are giving TW to start nodes
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot
    # and 'copy' the slack var in the solution object (aka Assignment) to print it
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == 0:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
        time_dimension.CumulVar(index).RemoveInterval(data['break_start_time'], data['break_end_time'])
        routing.AddToAssignment(time_dimension.SlackVar(index))
    # Add time window constraints for each vehicle start node
    # and 'copy' the slack var in the solution object (aka Assignment) to print it
    for vehicle_id in xrange(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(data['time_windows'][0][0],
                                                data['time_windows'][0][1])
        routing.AddToAssignment(time_dimension.SlackVar(index))
        # Warning: Slack var is not defined for vehicle's end node
        #routing.AddToAssignment(time_dimension.SlackVar(self.routing.End(vehicle_id)))


###########
# Printer #
###########
def print_solution(data, manager, routing, assignment):  # pylint:disable=too-many-locals
    """Prints assignment on console"""
    print('Objective: {}'.format(assignment.ObjectiveValue()))
        
    #Prints dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if assignment.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
    print(dropped_nodes, '\n')
    
    #prints breaks
    print('Breaks:')
    intervals = assignment.IntervalVarContainer()
    for i in xrange(intervals.Size()):
        brk = intervals.Element(i)
        if brk.PerformedValue() == 1:
            print('{}: Start({}) Duration({})'.format(
                brk.Var().Name(),
                brk.StartValue(),
                brk.DurationValue()))
        else:
            print('{}: Unperformed'.format(brk.Var().Name()))

    total_load = 0
    total_time = 0
    capacity_dimension = routing.GetDimensionOrDie('Capacity')
    time_dimension = routing.GetDimensionOrDie('Time')
    for vehicle_id in xrange(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            load_var = capacity_dimension.CumulVar(index)
            time_var = time_dimension.CumulVar(index)
            slack_var = time_dimension.SlackVar(index)
            plan_output += ' {0} Load({1}) Time({2},{3}) Slack({4},{5}) ->'.format(
                manager.IndexToNode(index),
                assignment.Value(load_var),
                assignment.Min(time_var),
                assignment.Max(time_var),
                assignment.Min(slack_var), assignment.Max(slack_var))
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
        load_var = capacity_dimension.CumulVar(index)
        time_var = time_dimension.CumulVar(index)
        slack_var = time_dimension.SlackVar(index)
        plan_output += ' {0} Load({1}) Time({2},{3})\n'.format(
            manager.IndexToNode(index),
            assignment.Value(load_var),
            assignment.Min(time_var), assignment.Max(time_var))
        plan_output += 'Load of the route: {}\n'.format(
            assignment.Value(load_var))
        plan_output += 'Time of the route: {}\n'.format(
            assignment.Value(time_var))
        print(plan_output)
        total_load += assignment.Value(load_var)
        total_time += assignment.Value(time_var)
    print('Total Load of all routes: {}'.format(total_load))
    print('Total Time of all routes: {0}min'.format(total_time))
    

def save_solution(data, manager, routing, assignment, output_path):  # pylint:disable=too-many-locals
    
    #sets up and saves the dropped nodes
    dropped_nodes_df = pd.DataFrame(columns = ['dropped_addresses'])
    
    """Prints assignment on console"""
    print('Objective: {}'.format(assignment.ObjectiveValue()))
        
    #Prints dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if assignment.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
            dropped_nodes_df = dropped_nodes_df.append({'dropped_addresses' : data['address_lookup'][node]}, ignore_index = True)
    print(dropped_nodes, '\n')
    print(dropped_nodes_df)
    
    #Saves Dropped Nodes
    
    #sets up the delivery vehicle & config csvs
    routing_list = []
    config_df = pd.DataFrame(columns = ['param','info'])
    
    #populates solver info
    config_df = config_df.append({'param': 'num_locations','info':data['num_locations']}, ignore_index = True)
    config_df = config_df.append({'param': 'service_time','info':data['time_per_demand_unit']}, ignore_index = True)
    config_df = config_df.append({'param': 'num_vehicles','info':data['num_vehicles']}, ignore_index = True)
    config_df = config_df.append({'param': 'vehicle_capacity','info':data['vehicle_capacity']}, ignore_index = True)
    config_df = config_df.append({'param': 'break_start_time','info':data['break_start_time']}, ignore_index = True)
    config_df = config_df.append({'param': 'break_end_time','info':data['break_end_time']}, ignore_index = True)
    config_df = config_df.append({'param': 'break_duration','info':data['break_duration']}, ignore_index = True)
    config_df = config_df.append({'param': 'solver_limit','info':data['solver_limit']}, ignore_index = True)
    config_df = config_df.append({'param': 'Objective_value','info':assignment.ObjectiveValue()}, ignore_index = True)
    
    
    #prints breaks
    print('Breaks:')
    intervals = assignment.IntervalVarContainer()
    for i in xrange(intervals.Size()):
        brk = intervals.Element(i)
        if brk.PerformedValue() == 1:
            print('{}: Start({}) Duration({})'.format(
                brk.Var().Name(),
                brk.StartValue(),
                brk.DurationValue()))
        else:
            print('{}: Unperformed'.format(brk.Var().Name()))

    total_load = 0
    total_time = 0
    capacity_dimension = routing.GetDimensionOrDie('Capacity')
    time_dimension = routing.GetDimensionOrDie('Time')
    
    
    for vehicle_id in xrange(data['num_vehicles']):
        #sets up save dataframe
        routing_df = pd.DataFrame(columns = ['address', 'arrival_time','time_slot_start','time_slot_end'])
        
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            load_var = capacity_dimension.CumulVar(index)
            time_var = time_dimension.CumulVar(index)
            slack_var = time_dimension.SlackVar(index)
            plan_output += ' {0} Load({1}) Time({2},{3}) Slack({4},{5}) ->'.format(
                manager.IndexToNode(index),
                assignment.Value(load_var),
                assignment.Min(time_var),
                assignment.Max(time_var),
                assignment.Min(slack_var), assignment.Max(slack_var))
            
            #appends driver details
            routing_df = routing_df.append({'address': data['address_lookup'][index] , 'arrival_time':assignment.Min(time_var),
                                           'time_slot_start': data['time_slot_lookup'][index][0],
                                           'time_slot_end': data['time_slot_lookup'][index][1],}, ignore_index = True)
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            
         
        load_var = capacity_dimension.CumulVar(index)
        time_var = time_dimension.CumulVar(index)
        slack_var = time_dimension.SlackVar(index)
        plan_output += ' {0} Load({1}) Time({2},{3})\n'.format(
            manager.IndexToNode(index),
            assignment.Value(load_var),
            assignment.Min(time_var), assignment.Max(time_var))
        plan_output += 'Load of the route: {}\n'.format(
            assignment.Value(load_var))
        plan_output += 'Time of the route: {}\n'.format(
            assignment.Value(time_var))
        
        # adds in the return depot location & departure time
        routing_df = routing_df.append({'address': data['address_lookup'][index] , 'arrival_time':assignment.Min(time_var),
                                       'time_slot_start': data['time_slot_lookup'][0][0],
                                           'time_slot_end': data['time_slot_lookup'][0][1]}, ignore_index = True)
        routing_df["next_address"] = routing_df["address"].shift(-1).fillna(data['address_lookup'][0])
        routing_df['latest_departure'] = 0
        routing_df['earliest_departure'] = routing_df['arrival_time'] + data['time_per_demand_unit']
        routing_df.at[0,'earliest_departure'] = 0
        routing_df["travel_time_to_next"] = 0
        
        #adds in latest departure time
        for index, row in routing_df.iterrows():
            if index < len(routing_df) - 1:
                routing_df.at[index, 'latest_departure'] = int(routing_df.at[index + 1, 'arrival_time']) - int(data['raw_time_matrix'][routing_df.at[index, 'next_address']][routing_df.at[index, 'address']])
                routing_df.at[index,'travel_time_to_next'] = int(data['raw_time_matrix'][routing_df.at[index, 'next_address']][routing_df.at[index, 'address']])
            else:
                routing_df.at[index, 'latest_departure'] = 0
                
        #adds in break
        break_start_list = routing_df[routing_df["arrival_time"] < data["break_start_time"]]["earliest_departure"].tolist()
        
        if len(break_start_list) > 0:
            break_start = break_start_list[len(break_start_list) - 1]
            break_end = break_start + data["break_duration"]
            break_index = routing_df[routing_df["arrival_time"] < data["break_start_time"]].index[len(break_start_list) - 1] + 0.5
            break_information = pd.DataFrame({'address': 'BREAK' , 'arrival_time':break_start,
                                              'latest_departure': break_end,'earliest_departure': break_end,
                                             'time_slot_start': break_start, 'time_slot_end': break_end},
                                             index = [break_index])

            routing_df = routing_df.append(break_information, ignore_index = False).sort_index().reset_index(drop = True)

        #formats solution in terms of time
        
        #list to convert
        convert_to_time = ['arrival_time','latest_departure','earliest_departure','time_slot_start','time_slot_end']
        for convert in convert_to_time:
            routing_df[convert] = (data['delivery_start_time'] + routing_df[convert].apply(lambda x: timedelta(minutes = x)))
            routing_df[convert] = pd.to_datetime(routing_df[convert]).dt.strftime('%H:%M')
        
        #reorders columns and appends to list
        col_order = ['address','time_slot_start','time_slot_end','arrival_time', 'earliest_departure', 'latest_departure']
        routing_list.append(routing_df[col_order])
        
        # prints solution
        print(plan_output)
        total_load += assignment.Value(load_var)
        total_time += assignment.Value(time_var)
        
        config_df = config_df.append({'param': f'driver_{vehicle_id}_total_time','info':assignment.Value(time_var)}, ignore_index = True)
        config_df = config_df.append({'param': f'driver_{vehicle_id}_total_load','info':assignment.Value(load_var)}, ignore_index = True)
        #print(routing_df)
        
        #Vehicle travel time and load
        #configurations
    print('Total Load of all routes: {}'.format(total_load))
    print('Total Time of all routes: {0}min'.format(total_time))
    
    config_df = config_df.append({'param': 'total_time','info':total_time}, ignore_index = True)
    config_df = config_df.append({'param': 'total_load','info':total_load}, ignore_index = True)
    
    #saves all dataframes 
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for i in range(len(routing_list)):
        routing_list[i].to_csv(os.path.join(output_path,f"driver_{i}_schedule.csv"))
        print(routing_list[i])
        
    config_df.to_csv(os.path.join(output_path,"config.csv"))
    dropped_nodes_df.to_csv(os.path.join(output_path,"dropped_nodes.csv"))



# TSP Solver

In [8]:
########
# Main #
########
"""Entry point of the program"""
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager
manager = pywrapcp.RoutingIndexManager(data['num_locations'],
                                       data['num_vehicles'], data['depot'])

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

# Define weight of each edge
time_evaluator_index = routing.RegisterTransitCallback(
    partial(create_time_evaluator(data), manager))
routing.SetArcCostEvaluatorOfAllVehicles(time_evaluator_index)

# Add Capacity constraint
demand_evaluator_index = routing.RegisterUnaryTransitCallback(
    partial(create_demand_evaluator(data), manager))
add_capacity_constraints(routing, data, demand_evaluator_index)

# Add Time Window constraint
time_evaluator_index = routing.RegisterTransitCallback(
    partial(create_time_evaluator(data), manager))
add_time_window_constraints(routing, manager, data, time_evaluator_index)

#     # Add breaks
#     time_dimension = routing.GetDimensionOrDie("Time")
#     node_visit_transit = {}
#     for n in xrange(routing.Size()):
#         if n >= data['num_locations']:
#             node_visit_transit[n] = 0
#         else:
#             node_visit_transit[n] = int(
#                 data['demands'][n])# * data['time_per_demand_unit'])

#     break_intervals = {}
#     #for v in range(data['num_vehicles']):
#     for v in [0]:
#         vehicle_break = data['breaks'][v]
#         break_intervals[v] = [
#             routing.solver().FixedDurationIntervalVar(
#                 135, 435, vehicle_break, False, 'Break for vehicle {}'.format(v))
#             ]
#         time_dimension.SetBreakIntervalsOfVehicle(
#             break_intervals[v], v, node_visit_transit)


# Allow to drop nodes.
penalty = 999999999
for node in range(1, len(data['time_matrix'])):
    routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

# Setting first solution heuristic (cheapest addition).
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)  # pylint: disable=no-member
search_parameters.log_search = True
search_parameters.local_search_metaheuristic = (
   routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.solution_limit = data['solver_limit'] 


# Solve the problem.
assignment = routing.SolveWithParameters(search_parameters)
if(assignment):
    print_solution(data, manager, routing, assignment)
else:
    print("No Solutions Found")

Objective: 468
Dropped nodes: 

Breaks:
Route for vehicle 0:
 0 Load(0) Time(0,118) Slack(0,118) -> 23 Load(0) Time(15,130) Slack(0,115) -> 18 Load(1) Time(135,154) Slack(0,19) -> 6 Load(2) Time(148,167) Slack(0,19) -> 17 Load(3) Time(160,179) Slack(0,19) -> 4 Load(4) Time(172,191) Slack(0,19) -> 14 Load(5) Time(194,213) Slack(0,19) -> 5 Load(6) Time(206,225) Slack(0,19) -> 3 Load(7) Time(222,241) Slack(0,19) -> 28 Load(8) Time(235,254) Slack(49,113) -> 9 Load(9) Time(316,361) Slack(0,45) -> 26 Load(10) Time(335,380) Slack(0,45) -> 24 Load(11) Time(353,398) Slack(0,45) -> 10 Load(12) Time(367,412) Slack(0,45) -> 7 Load(13) Time(390,435) Slack(0,999597) -> 0 Load(14) Time(402,999999)
Load of the route: 14
Time of the route: 402

Route for vehicle 1:
 0 Load(0) Time(0,72) Slack(0,72) -> 20 Load(0) Time(15,80) Slack(0,65) -> 12 Load(1) Time(28,93) Slack(0,65) -> 27 Load(2) Time(42,107) Slack(0,65) -> 15 Load(3) Time(56,121) Slack(0,65) -> 2 Load(4) Time(135,137) Slack(0,2) -> 1 Load(5) Ti

In [9]:
save_solution(data, manager, routing, assignment, output_path)

Objective: 468
Dropped nodes: 

Empty DataFrame
Columns: [dropped_addresses]
Index: []
Breaks:
Route for vehicle 0:
 0 Load(0) Time(0,118) Slack(0,118) -> 23 Load(0) Time(15,130) Slack(0,115) -> 18 Load(1) Time(135,154) Slack(0,19) -> 6 Load(2) Time(148,167) Slack(0,19) -> 17 Load(3) Time(160,179) Slack(0,19) -> 4 Load(4) Time(172,191) Slack(0,19) -> 14 Load(5) Time(194,213) Slack(0,19) -> 5 Load(6) Time(206,225) Slack(0,19) -> 3 Load(7) Time(222,241) Slack(0,19) -> 28 Load(8) Time(235,254) Slack(49,113) -> 9 Load(9) Time(316,361) Slack(0,45) -> 26 Load(10) Time(335,380) Slack(0,45) -> 24 Load(11) Time(353,398) Slack(0,45) -> 10 Load(12) Time(367,412) Slack(0,45) -> 7 Load(13) Time(390,435) Slack(0,999597) -> 0 Load(14) Time(402,999999)
Load of the route: 14
Time of the route: 402

Route for vehicle 1:
 0 Load(0) Time(0,72) Slack(0,72) -> 20 Load(0) Time(15,80) Slack(0,65) -> 12 Load(1) Time(28,93) Slack(0,65) -> 27 Load(2) Time(42,107) Slack(0,65) -> 15 Load(3) Time(56,121) Slack(0,65